# Market - > Features Engineering

In [1]:
path = '../scripts'
import sys
sys.path.append(path)
from market_imports import *
from market_code import *

data_path = '/Users/jacob/Desktop/docs/kaggle/two_sigma/_g_data/data'
df_market = pd.read_csv(data_path + '/marketdata_sample.csv') 
df_news = pd.read_csv(data_path + '/news_sample.csv')

In [3]:
# LIST OF ASSETS PULLED FROM KERNEL 
# with open('output/asset_kernel.txt', 'w') as file:
#     file.write(str(assets))
    

In [ ]:
# TRYING TO PREVENT DATE TYPE CONVERSION
# THIS SHOULD BE UPDATED FROM PANDA DATAREADER
csv = pd.read_csv(data_path + '/sigma_data.csv', index_col='Date', parse_dates=True, 
                 infer_datetime_format=True)

print(csv.shape)

check = csv.columns[:4].tolist()
csv.drop_duplicates(check, inplace=True)

print(csv.shape)

In [ ]:
# COMPUTE RESIDUAL FOR RESPONSE
import statsmodels.tsa.api as tsa
from statsmodels.tsa.seasonal import seasonal_decompose 

x = csv[['returns_open_raw10_next']]
# Freq will depend on asset number - this needs to be researched
result = seasonal_decompose(x, model='additive', freq=50)

csv['returns_res10_next'] = result.resid
# Lose the first and last day
csv = csv[-csv['returns_res10_next'].isnull()]

In [ ]:
csv.Close.describe()

In [ ]:
csv.Close.max()

In [ ]:
# ELC is insanely high. The next highest price is around 300
csv[csv.Close ==csv.Close.max()]
csv[csv.Close > 1000]
csv = csv[csv.asset != 'ELC']

In [ ]:
data = csv.copy()

### Sample set

In [ ]:
# IF YOU WANT TO UPDATE

# with open('/Users/jacob/Desktop/docs/kaggle/two_sigma/_g_data/output/asset.txt',
#          'r') as file:
#     assets = file.read()
# file.close()  
##################################

# assets = ['CHL.N', 'RAD.N', 'HBC', 'PDNT.O', 'IGLD.O', '0857.F', 'GOOGa.DE', 'DIS.N', '1285.HK', 'DIS.F', 'MTTX.O', 'NDAQ.O', 'ALBK.F', 'AAPL.OQ', 'GS.N', 'WF.N', 'HTX.N', 'UPM1V.HE', 'GE.N', 'LPL.N', 'RYAAY.OQ', 'MHR.A', 'KB.N', '1301.HK', 'YHOO.O', 'SRIB.O', 'RYAAY.O', '2332.DE', 'YHOO.OQ', 'TRV.DE', '0941.HK', 'SIRI.O', 'UPM.N', 'SIEB.O', 'DWMT.O', 'NGG.N', '3777.HK', 'NML.A', 'NWS.N', 'SAN.MC', 'PGEB.O', 'MER.N', 'DSP.A', 'MON.DE', 'NOKy.DE', 'ADBE.DE', 'MTTT.O', 'SI.N', '3838.HK', 'NG.L', 'NOKS.DE', 'QCOM.DE', 'DIS.DE', 'CS.AS', 'TSM.N', 'C', 'MTNK.O', 'AAPL.F', '0005.HK', 'QCOM.O', 'IGLD.TA', 'ADBE.OQ', 'SIEB.OQ', 'STZb.N', 'AIB.N', 'BEN.N', 'IGLD.OQ', 'SNE.N', 'TWX.N', 'DCX.N', 'THC.N', 'MSFT.DE', 'NWSa.N', 'ISAT.JK', '0857.HK', '053000.KS', 'NWS.AX', 'WMT.DE', 'AUO.N', 'SIEGnq.L', 'KEP.N', 'BHP.F', 'ELC.A', '2332.HK', 'AAPL.DE', 'NOK1V.HE', 'WSH.N', '2330.TW', 'MX.N', 'NOKS.F', 'XMSR.O', 'FMSAUBD.MX', 'BHP.N', 'SIRI.OQ', 'PLJC.O', 'MSPX.O', 'NOK.N', 'YHOO.DE', 'CHLy.DE', 'DCXGn.F', 'SIEGn.DE', 'STZ.N', '0941.F', 'STD.BA', 'ALBK.I', '015760.KS', 'STD.N', 'VIAb.N', 'CSNA3.SA', '7267.T', 'MSFT.O', 'C.N', 'AAPL.O', '0941.DE', 'MNNY.O', 'RBS.L', 'TIE.N', 'HMC.N', '6758.T', 'HSBA.L', 'NDAQ.OQ', 'ALBK.L', 'MTSM.O', 'EST.A', 'IIT.N', 'WMT.N', '0857.DE', 'VIA.N', 'RBV.N', 'MLN.P', 'BHP.AX', 'CTCH.O', 'RBS.DE', 'CGA.N', 'MON.F', 'CS.L', 'GOOG.OQ', 'MON.N', 'SASY.PA', 'MEO1V.HE', 'SIEGn.F', '2409.TW', 'DCXGn.DE', 'STA.N', 'SASY.F', 'ADBE.O', 'SNY.N', 'RBS', 'RYA.I', '034220.KS', 'RYA.L', 'MSFT.OQ', 'RYA.F', 'MSFT.F', 'HBC.N', 'VNV.N', 'HBC.PA', 'FMX.N', 'QCOM.OQ', '060000.KS', 'TRV.F', 'GOOG.O', 'PTR.N', 'SID.N']
# sub_asst = assets[:200]

# sub = [i.split('.')[0] for i in sub_asst]     

# data = marketDataGen(sub)

# data.to_csv(data_path + '/sigma_data.csv')

In [ ]:
test = data[data.asset.isin(['AAPL'])]
test = test[['asset', 'Close']]
test.head()

### Moving Averages

In [ ]:
data = csv.copy()

data['ema_26'] = data.groupby('asset')['Close'].transform(lambda x: x.ewm(span=26, 
                                                                          adjust=False).mean())

data['ema_12'] = data.groupby('asset')['Close'].transform(lambda x: x.ewm(span=12, 
                                                                          adjust=False).mean())


data['ema_9'] = data.groupby('asset')['Close'].transform(lambda x: x.ewm(span=9, 
                                                                          adjust=False).mean())

data.tail()

In [ ]:
# CONVERT TO FUNC
# scored pretty low on importance. sma_20 is included for bollinger. Thats enough for now. Could consider a crossover
# indicator
data = csv.copy()

spans = [5, 20, 50]
for span in spans:
    print(span)
    data['sma_' + str(span)] = data.groupby('asset')['Close'].transform(lambda x: x.rolling(window=span).mean())
data.tail()

### MACD 

If correlation becomes an issue, could consider crossover indicators easy enough  



Moving average convergence divergence (MACD) is a trend-following momentum indicator that shows the relationship between two moving averages of prices. The MACD is calculated by subtracting the 26-day exponential moving average (EMA) from the 12-day EMA. A nine-day EMA of the MACD, called the "signal line", is then plotted on top of the MACD, functioning as a trigger for buy and sell signals.

MACD can be interpreted using 3 different methods:

__Crossover__: When the MACD falls bellow the signal line (9-day EMA) this is a bearish signal. When the MACD rise above the signal line, this is a bulish signal.

__Divergence__: When the price diverges from MACD, it signal the end of the current trend

__Dramatic Rise__: When MACD rises dramatically, that is, the shorter moving average pulls away from the longer-term moving average, this is a signal that the stock is overbought

Source: https://www.investopedia.com/terms/m/macd.asp



In [ ]:
# MACD - MANUAL
data = csv.copy() 

ema_26 = data.groupby('asset')['Close'].transform(lambda x: x.ewm(span=26).mean())
ema_12 = data.groupby('asset')['Close'].transform(lambda x: x.ewm(span=12).mean())
data['macd'] = ema_12 - ema_26

data['signal'] = data.groupby('asset')['macd'].transform(lambda x: x.ewm(span=9).mean())

data.tail()

In [ ]:
# MACD - BETTER FORM
data = csv.copy() 

def macd(dframe):
    
    asset_close = dframe.groupby('asset')['Close']
    ema_26 = asset_close.transform(lambda x: x.ewm(span=26).mean())
    ema_12 = asset_close.transform(lambda x: x.ewm(span=12).mean())
    
    return ema_12 - ema_26

data['macd'] = macd(data)

data['signal'] = (
                data.groupby('asset')['macd'].
                transform(lambda x: x.ewm(span=9).
                mean() )
                )

data.tail()

In [ ]:
data = csv.copy() 

def macd(dframe):
    
    """
    If macd is negative/positive, the shorter term mv avg is below/above the longer 
    term and momentum is downward/upward - ish.
    
    If the signal is simply the ema of macd. If macd is less/greater than the signal,
    whether pos or neg, the momentum is bear/bullish
    
    Ex
    -----
    both signal and macd are negative. However, macd is noticably less
    negative than the signal. Interpretation: the asset has been in a short term down
    trend (short term down trend could be part of a long term down/up trend). However, 
    the momentum is bullish
    
    Metrics for indicators
    ------
    Crossover: macd is below/above signal
    
    Divergence: the price diverges by a certain threshold from macd. End of 
    current trend
    
    Dramatic Rise: macd is significantly above signal - overbought
       
    """
    
    asset_close = dframe.groupby('asset')['Close']
    ema_26 = asset_close.transform(lambda x: x.ewm(span=26).mean())
    ema_12 = asset_close.transform(lambda x: x.ewm(span=12).mean())
    # switch adjusted back to default
    
    return ema_12 - ema_26



data['macd'] = macd(data)

data['signal'] = (
                data.groupby('asset')['macd'].
                transform(lambda x: x.ewm(span=9).
                mean() )
                )



In [ ]:
# CONVERT TO ARRAYS
macd = data['macd'].values
sig = data['signal'].values
close = data['Close'].values
diff = macd - sig # dramatic rise
div = abs(close - macd) # diverge

# VECTORIZED (FASTER) -> SET AS BOOLS FOR PIPELINES
data['macd_cross'] = np.where(macd > sig, 1, 0).astype('bool')
data['macd_sharp_rise'] = np.where(diff > 0.09, 1, 0).astype('bool')
data['macd_div'] = np.where(div > 75, 1, 0).astype('bool')

data.tail()

In [ ]:
# PRETTY LOUSY SUMMARY STATS FROM NUMPY. PUT IN PANDAS FOR STATS.
# definitely some serious outliers. Need to think about outlier reduction

import scipy
scipy.stats.describe(div)

In [ ]:
# USE QUARTILES FOR NOW. IF IN THE UPPER 80%, THEN CLASSIFY AS DIVERGING. SAME THING FOR DRAMATIC RISE. MEAN AND STD
# can't be trusted until the outliers are dealt with
pd.DataFrame(div).describe()

### Bollinger Band  

https://www.investopedia.com/articles/technical/04/030304.asp  



Equities alternate between periods of low volatility and high volatility–much like the calm before the storm and the inevitable activity afterward.

__squeeze__ $= \frac{UB - LB}{SMA}$  

Squeeze candidate is identified as a 6 month low. So, add the feature and then add a 6 month trailing minimum indicator



In [ ]:
# MANUAL
data = csv.copy() 

sd_20 = df.groupby('asset')['Close'].transform(lambda x: x.rolling(window=20).std())

sma_20 = df.groupby('asset')['Close'].transform(lambda x: x.rolling(window=20).mean())

df['BB_upper'] = sma_20 + (sd_20*2)

df['BB_lower'] = sma_20 - (sd_20*2)

                                    
df.tail()

In [ ]:
data = csv.copy() 

def BB(dframe):
    
    
    asset_close = dframe.groupby('asset')['Close']
    
    sd_20 = (
            asset_close.
            transform( lambda x: x.rolling(window=20).
            std() )
            )

    sma_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            mean())
            )

    
    return ( sma_20 + (sd_20*2), sma_20 - (sd_20*2) )


data['BB_upper'], data['BB_lower'] = BB(data)

                                    
data.tail()

In [ ]:
data = csv.copy() 

def BB_distance(dframe):
    
    '''price moves toward upper indicate bullish. This
    returns the distance from stock price to upper or lower 
    BB. If the distance turns negative, the price is above/below. 
    See indicator lambdas.
    
     Squeeze is calculate per asset. However, comparing it to the 6 month min is a bit
    more involved. For now, leave it external. It can be integrated into the func/class
    later if it is useful. Notice the shift on the rolling min(). The present squeeze value
    should not be included in the window we are comparing it with.
    '''
    
    asset_close = dframe.groupby('asset')['Close']
    
    sd_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            std()).
            values        
            )
    
    sma_20 = (
            asset_close.
            transform(lambda x: x.rolling(window=20).
            mean()).
            values
            )
    
    # convert everything to arrays
    
    close = dframe['Close'].values   
    U = sma_20 + (sd_20*2)    
    L = sma_20 - (sd_20*2) 
    squeeze = (U - L)/ sma_20

    
#     return (U - close, close - L, squeeze)
    return (U, L, squeeze)

U, L,  squeeze = BB_distance(data)
    
    
# data['BB_upper_dis'], data['BB_lower_dis'],  data['squeeze']= BB_distance(data)
    
    
data.tail()

In [ ]:
data = csv.copy()
# A SHIFT IS INCLUDED TO EXCLUDE THE CURRENT VALUE TO BE COMPARED
data['squeeze'] = squeeze
sq_min = (
        data.groupby('asset')['squeeze'].
        transform(lambda x: x.rolling(window=126).
        min().
        shift())
        )

data['low_vol'] = np.where(squeeze <= sq_min, 1, 0).astype('bool')
# data['min'] = sq_min
data.tail()

In [ ]:
# LOOKS LIKE ANY DISTANCE BELOW 0.3 IS LESS THAN 20%

C = data['Close'].values
data['Bollinger_bearish'] = np.where(C-L <= 0.3, 1, 0).astype('bool')
data['Bollinger_bullish'] = np.where(U-C <= 0.3, 1, 0).astype('bool')

data.tail()

### RSI  

Original code. Walthrough (single asset), then custom function with groupby/transform/lambda. Time on 4M + set decent. Could look to vectorize later.

In [ ]:
def relative_strength_index(close, n):
    """Calculate Relative Strength Index(RSI) for given data.
    
    :param df: pandas.Series
    :param n: 
    :return: pandas.DataFrame
    """
    buf = pd.DataFrame()
    buf['close'] = close
    buf['diff'] = buf.close.diff()
    mask = buf['diff'] < 0
    buf['high'] = abs(buf['diff'].mask(mask))
    buf['low'] = abs(buf['diff'].mask(~mask))
    buf['high'] = buf['high'].fillna(0)
    buf['low'] = buf['low'].fillna(0)
    posrs = buf['high'].ewm(span=n, min_periods=n).mean()
    negrs = buf['low'].ewm(span=n, min_periods=n).mean()
    buf['rsi'] =  posrs / (posrs + negrs)
    return buf.rsi
    

In [ ]:
# MANUAL METHOD -> ONE ASSET AT A TIME (OR ELSE IT WILL MERGE EVERYTHING)
test['diff'] = test.Close.diff()
mask = test['diff'] < 0
test['high'] = abs(test['diff'].mask(mask)).fillna(0)
test['low'] = abs(test['diff'].mask(~mask)).fillna(0)
pos_rsi = test['high'].ewm(span=14, min_periods=14).mean()
neg_rsi = test['low'].ewm(span=14, min_periods=14).mean()
test['rsi'] =  pos_rsi / (pos_rsi + neg_rsi)

In [ ]:
test.tail()

In [ ]:
# FUNCTION METHOD - single asset
# simply replacing cols with unassigned panda series
# cannot imagine this is very fast
diff = test.Close.diff()
mask = diff < 0
high = abs(diff.mask(mask)).fillna(0)
low = abs(diff.mask(~mask)).fillna(0)
pos_rsi = high.ewm(span=14).mean()
neg_rsi = low.ewm(span=14).mean()
test['rsi'] =  pos_rsi / (pos_rsi + neg_rsi)


In [ ]:
# SET UP FOR A LAMBDA COLUMN TRANSFORM
# as used, the incoming vec is the asset.Close

def rsi(x):
    
    diff = x.diff()
    mask = diff < 0
    high = abs(diff.mask(mask)).fillna(0)
    low = abs(diff.mask(~mask)).fillna(0)
    pos_rsi = high.ewm(span=14).mean()
    neg_rsi = low.ewm(span=14).mean()
    return  pos_rsi / (pos_rsi + neg_rsi)


# TESTED WITH 2 ASSETS. SEEMS TO WORK. CLOSE IS PULLED FROM THE FUNCTION
# the x is the asset.Close
test['rsi'] = test.groupby('asset')['Close'].transform(lambda x: rsi(x))


In [ ]:
test.tail()